In [1]:
import os, sys, math, random, time
from datetime import datetime
import _pickle as pickle

import numpy as np
import pandas as pd

import mxnet as mx
from mxnet import autograd, gluon, nd, gpu
from mxnet.gluon import nn

from newModels import TCN
from newTrainer import nn_trainer

In [2]:
### The input dataset
ctx = mx.gpu(2)
with open('feature_prepare.pkl', 'rb') as f:
    [trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt] = pickle.load(f)

In [3]:
sub_train_X, sub_train_Y = nd.array(trainX_dt, ctx=ctx), nd.array(trainY_dt, ctx=ctx)
sub_valid_X, sub_valid_Y = nd.array(testX_dt, ctx=ctx), nd.array(testY_dt, ctx=ctx)
future_train_X, future_test_X = nd.array(trainY2_dt, ctx=ctx), nd.array(testY2_dt, ctx=ctx)

weight = future_train_X[:,0:1,7]+3
sub_train_nd = gluon.data.ArrayDataset(sub_train_X, future_train_X, sub_train_Y,weight)

In [4]:
model1 = TCN()
#choose parameters
batch_size= 512
n_epochs=500
"""
The model training
"""
### The model parameters
abs_loss = gluon.loss.L1Loss()
L2_loss = gluon.loss.L2Loss()
huber_loss = gluon.loss.HuberLoss()
initializer = mx.initializer.MSRAPrelu()
optimizer = 'adam';
optimizer_params = {'learning_rate': 0.05}

trainer_params_list = {'batch_size': batch_size,'epoch_num':n_epochs,
                'loss_func': huber_loss, 'initializer': initializer,
                'optimizer':optimizer, 'optimizer_params':optimizer_params}
train_mark='testing'
nn_trainer(train_mark, model1, sub_train_nd, sub_valid_X,future_test_X, sub_valid_Y, trainer_params_list=trainer_params_list, ctx=ctx)

  0%|          | 0/500 [00:00<?, ?it/s]

(2590, 24)
[12.690355  11.42132   11.42132   11.42132   12.690355  11.42132
 11.42132   11.42132    5.0761423 10.152285   6.3451777  7.614213
  6.3451777  6.3451777  6.3451777 15.228426  11.42132    7.614213
  8.883248   7.614213   8.883248  10.152285   8.883248   8.883248 ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Epoch 0, valid loss: 5140082.000000 valid ND: 1.000000, valid NRMSE 4.632637
Epoch 0, valid loss: 5140082.000000 valid ND: 1.000000, valid NRMSE 4.582995
Epoch 0, valid loss: 5140082.000000 valid rho-risk 50: 1.000000,  valid rho-risk 90: 1.800000
----------------------------------The group result-------------------------------------
 average ND: 1.000000, valid NRMSE 1.01.0-1.0--1.0-1.0-1.0-1.0
 average NRMSE: 4.612024, valid NRMSE 4.4854284.436387-4.5919566--4.6564164-4.569165-4.522523-5.0222945
 average risk50: 1.000000, valid risk50 1.00000008701184081.0000000169464367-0.9999999180468667--0.9999999781606804-1.0000000515154435-1.0000000065

  0%|          | 1/500 [00:25<3:31:58, 25.49s/it]


KeyboardInterrupt: 